In [1]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [12]:
import matplotlib
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [13]:
# Confirm that patient sheet imported correctly
patients.head()

,SFM Id,Gender,BirthYear,City,State,Country,FitzpatrickSkinPhotoType,SkinTone,SkinConditions
0,10918,M,2000,Sunnyvale,California,United States of America,,dark,none
1,16531,F,1946,Pender,Nebraska,United States of America,,medium,"sensitive-skin-allergist-diagnosed,sensitive-..."
2,23284,F,1950,Irvine,Kentucky,United States of America,,fair,"sensitive-skin-allergist-diagnosed, eczema-at..."
3,25628,M,1953,,,,,fair,"sensitive-skin-allergist-diagnosed,skin-aller..."
4,29136,F,1985,Cambridge,Massachusetts,United States of America,,fair,"sensitive-skin-allergist-diagnosed, sensitive..."


In [14]:
# Confirm allergy sheet imported correctly
allergies.head()

,SFM Id,Allergens,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98
0,10918,Ingredient 5443,Ingredient 15438,Ingredient 18419,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16531,Ingredient 9456,Ingredient 1613,Ingredient 201,Ingredient 2023,Ingredient 2787,Ingredient 1600,Ingredient 9315,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23284,Ingredient 1607,Ingredient 1462,Ingredient 144962,Ingredient 188,Ingredient 1827,Ingredient 5986,Ingredient 1590,Ingredient 1600,Ingredient 124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25628,Ingredient 214,Ingredient 201,Ingredient 215,Ingredient 1605,Ingredient 3793,Ingredient 800,Ingredient 5467,Ingredient 3565,Ingredient 204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29136,Ingredient 201,Ingredient 215,Ingredient 257,Ingredient 263,Ingredient 1598,Ingredient 1605,Ingredient 1805,Ingredient 2023,Ingredient 2350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Merge columns by ID
patientAllergies = patients.merge(allergies, on = "SFM Id")
patientAllergies.head()

,SFM Id,Gender,BirthYear,City,State,Country,FitzpatrickSkinPhotoType,SkinTone,SkinConditions,Allergens,...,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98
0,10918,M,2000,Sunnyvale,California,United States of America,,dark,none,Ingredient 5443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16531,F,1946,Pender,Nebraska,United States of America,,medium,"sensitive-skin-allergist-diagnosed,sensitive-...",Ingredient 9456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23284,F,1950,Irvine,Kentucky,United States of America,,fair,"sensitive-skin-allergist-diagnosed, eczema-at...",Ingredient 1607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25628,M,1953,,,,,fair,"sensitive-skin-allergist-diagnosed,skin-aller...",Ingredient 214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29136,F,1985,Cambridge,Massachusetts,United States of America,,fair,"sensitive-skin-allergist-diagnosed, sensitive...",Ingredient 201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# TODO: Add layers/sequences to actually make the model 
model = Sequential()

# Use cross-entropy to predict loss for model and accuracy to get score
# TODO: Determine if loss function is needed (probably to allow for maximum efficiency, but should be fact-checked)
model.compile(loss = 'categorical_crossentropy',     
   optimizer = RMSprop(), 
   metrics = ['accuracy'])

In [ ]:
# TODO: create a training/testing split for the data

In [ ]:
# TODO: Change hyperparameters of the model to train/test
history = model.fit(
   x_train, y_train, 
   batch_size = 128, 
   epochs = 20, 
   verbose = 1, 
   validation_data = (x_test, y_test)
)

In [ ]:
# Get the average prediction score across all epochs
score = model.evaluate(x_test, y_test, verbose = 0)
score #First number is loss, second is accuracy

In [ ]:
# TODO: Save Keras model as separate file